In [ ]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import spacy
from tqdm import tqdm  # Import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from sklearn.metrics import accuracy_score, classification_report

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Load your dataset (replace 'your_dataset.csv' with the actual file path)
df = pd.read_csv(r'E:\SNU Chennai\projects\NLP project\NEWS classification\news article\news-article-categories.csv')

# Combine 'title' and 'body' columns into a single 'text' column
df['text'] = df['title'] + " " + df['body']

# Drop rows where 'text' is missing
df = df.dropna(subset=['text'])

# Preprocessing function using spaCy
def preprocess_text(text):
    # Process text with spaCy
    doc = nlp(text.lower())
    
    # Remove stopwords and non-alphabetic tokens, then lemmatize
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    # Join back into a string
    return " ".join(tokens)

# Apply preprocessing to the 'text' column with tqdm
print("Preprocessing text...")
tqdm.pandas()  # Enable tqdm progress_apply
df['clean_text'] = df['text'].progress_apply(preprocess_text)  # Use progress_apply instead of apply

# Encode labels
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Load pre-trained BERT tokenizer and model
print("Loading pre-trained BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Function to tokenize and encode text
def encode_text(texts, tokenizer, max_length=512):
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    return encodings

# Custom Dataset class
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Define the combined BERT + BiLSTM model
class BERTBiLSTM(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTBiLSTM, self).__init__()
        self.bert = bert_model
        self.bilstm = nn.LSTM(input_size=768, hidden_size=64, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128, num_classes)  # 128 because BiLSTM has 2 directions * 64 hidden units

    def forward(self, input_ids, attention_mask):
        # Get BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, 768)

        # Pass through BiLSTM
        lstm_out, _ = self.bilstm(bert_embeddings)  # Shape: (batch_size, seq_len, 128)
        lstm_out = lstm_out[:, -1, :]  # Take the last time step output: (batch_size, 128)

        # Dropout and fully connected layer
        out = self.dropout(lstm_out)
        logits = self.fc(out)  # Shape: (batch_size, num_classes)
        return logits

# Function for 5-fold cross-validation
def cross_validate(df, tokenizer, label_encoder, device, num_folds=5, epochs=100):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        print(f"\nFold {fold + 1}/{num_folds}")

        # Split data into train and validation sets
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]

        # Tokenize training and validation data
        train_encodings = encode_text(train_df['clean_text'].tolist(), tokenizer)
        val_encodings = encode_text(val_df['clean_text'].tolist(), tokenizer)

        # Convert labels to PyTorch tensors
        y_train_tensor = torch.tensor(train_df['category_encoded'].values, dtype=torch.long).to(device)
        y_val_tensor = torch.tensor(val_df['category_encoded'].values, dtype=torch.long).to(device)

        # Create datasets and dataloaders
        train_dataset = NewsDataset(train_encodings, y_train_tensor)
        val_dataset = NewsDataset(val_encodings, y_val_tensor)

        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

        # Initialize the model
        num_classes = len(label_encoder.classes_)
        model = BERTBiLSTM(bert_model, num_classes).to(device)

        # Unfreeze the last few layers of BERT for fine-tuning
        for name, param in model.bert.named_parameters():
            if 'layer.11' in name or 'pooler' in name:  # Unfreeze only the last layer and pooler
                param.requires_grad = True
            else:
                param.requires_grad = False

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=2e-5)

        # Train the model
        print("Training SBERT + BiLSTM model...")
        train_model(model, train_loader, criterion, optimizer, device, epochs=epochs)

        # Evaluate the model
        accuracy, report = evaluate_model(model, val_loader, device)
        print(f"\nFold {fold + 1} Accuracy: {accuracy:.4f}")
        print("\nClassification Report:\n", report)

        fold_accuracies.append(accuracy)

    print("\nCross-Validation Results:")
    print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
    print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")

# Training loop
def train_model(model, train_loader, criterion, optimizer, device, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_)
    return accuracy, report

# Perform 5-fold cross-validation
cross_validate(df, tokenizer, label_encoder, device, num_folds=5, epochs=100)

Using device: cuda
Preprocessing text...


100%|██████████| 6872/6872 [14:07<00:00,  8.11it/s] 


Loading pre-trained BERT model...

Fold 1/5
Training SBERT + BiLSTM model...


Epoch 1/100: 100%|██████████| 344/344 [00:17<00:00, 19.47it/s]


Epoch 1, Loss: 2.2610


...

Epoch 99, Loss: 0.0061


Epoch 100/100: 100%|██████████| 344/344 [00:17<00:00, 20.08it/s]


Epoch 100, Loss: 0.0042


Evaluating: 100%|██████████| 86/86 [00:03<00:00, 24.74it/s]


Fold 5 Accuracy: 0.9964

Classification Report:
                 precision    recall  f1-score   support

ARTS & CULTURE       1.00      1.00      1.00       185
      BUSINESS       1.00      0.95      0.98        83
        COMEDY       1.00      1.00      1.00        76
         CRIME       0.99      1.00      0.99        66
     EDUCATION       1.00      1.00      1.00        85
 ENTERTAINMENT       1.00      1.00      1.00       122
   ENVIRONMENT       1.00      1.00      1.00       113
         MEDIA       1.00      1.00      1.00        59
      POLITICS       1.00      1.00      1.00        99
      RELIGION       0.99      1.00      1.00       104
       SCIENCE       1.00      1.00      1.00        80
        SPORTS       1.00      1.00      1.00        99
          TECH       0.97      1.00      0.99       109
         WOMEN       1.00      0.99      0.99        94

      accuracy                           1.00      1374
     macro avg       1.00      1.00      1.00      1